# Medium Grid Chain

In [1]:
import random

import matplotlib.pyplot as plt
from functools import partial
import networkx as nx

from gerrychain import MarkovChain
from gerrychain.constraints import (
    Validator,
    single_flip_contiguous,
    within_percent_of_ideal_population,
)
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
from gerrychain.updaters import Tally, cut_edges
from gerrychain.partition import Partition
from gerrychain.proposals import recom
from numpy import linalg as LA

## Setup

### Create graph

In [2]:
gn = 6
k = 5
ns = 50
p = 0.5

graph = nx.grid_graph([k * gn, k * gn])

for n in graph.nodes():
    graph.node[n]["population"] = 1

    if random.random() < p:
        graph.node[n]["pink"] = 1
        graph.node[n]["purple"] = 0
    else:
        graph.node[n]["pink"] = 0
        graph.node[n]["purple"] = 1
    if 0 in n or k * gn - 1 in n:
        graph.node[n]["boundary_node"] = True
        graph.node[n]["boundary_perim"] = 1

    else:
        graph.node[n]["boundary_node"] = False

# this part adds queen adjacency
# for i in range(k * gn - 1):
#     for j in range(k * gn):
#         if j < (k * gn - 1):
#             graph.add_edge((i, j), (i + 1, j + 1))
#             graph[(i, j)][(i + 1, j + 1)]["shared_perim"] = 0
#         if j > 0:
#             graph.add_edge((i, j), (i + 1, j - 1))
#             graph[(i, j)][(i + 1, j - 1)]["shared_perim"] = 0

### Build assignment

In [3]:
cddict = {x: int(x[0] / gn) for x in graph.nodes()}
pos = {x: x for x in graph.nodes()}

### Configure updaters

In [4]:
def step_num(partition):
    parent = partition.parent
    if not parent:
        return 0
    return parent["step_num"] + 1


updaters = {
    "population": Tally("population"),
    "cut_edges": cut_edges,
    "step_num": step_num,
    # "Pink-Purple": Election("Pink-Purple", {"Pink":"pink","Purple":"purple"})
}

## First Markov chain

### Build first partition

In [5]:
grid_partition = Partition(graph, assignment=cddict, updaters=updaters)

### Add constraints

In [6]:
popbound = within_percent_of_ideal_population(grid_partition, 0.1)

### Setup Proposal

In [7]:
ideal_population = sum(grid_partition["population"].values()) / len(grid_partition)

tree_proposal = partial(
    recom,
    pop_col="population",
    pop_target=ideal_population,
    epsilon=0.05,
    node_repeats=1,
)

### Build and run first Markov chain

In [8]:
recom_chain = MarkovChain(
    tree_proposal,
    constraints=[popbound],
    accept=always_accept,
    initial_state=grid_partition,
    total_steps=100,
)

In [9]:
for part in recom_chain:
    pass

final_assignment = part.assignment

plt.figure()
nx.draw(
    graph,
    pos={x: x for x in graph.nodes()},
    node_color=[final_assignment[x] for x in graph.nodes()],
    node_size=ns,
    node_shape="s",
    cmap="tab20",
)
plt.savefig("./plots/medium/end_of_tree.png")
plt.close()
print("Finished ReCom")


Finished ReCom


## Second Markov chain

### Build second initial partition

In [10]:
squiggle_partition = Partition(graph, assignment=part.assignment, updaters=updaters)

### Add constraints

In [11]:
popbound = within_percent_of_ideal_population(squiggle_partition, 0.1)

### Build and run second chain

In [12]:
squiggle_chain = MarkovChain(
    propose_random_flip,
    Validator([single_flip_contiguous, popbound]),
    accept=always_accept,
    initial_state=squiggle_partition,
    total_steps=100_000,
)


for part2 in squiggle_chain:
    pass


plt.figure()
nx.draw(
    graph,
    pos={x: x for x in graph.nodes()},
    node_color=[part2.assignment[x] for x in graph.nodes()],
    node_size=ns,
    node_shape="s",
    cmap="tab20",
)
plt.savefig("./plots/medium/end_of_boundary.png")
plt.close()
print("Finished Squiggling")

Finished Squiggling


## Final Markov chain

### Build final partition

In [13]:
final_partition = Partition(graph, assignment=part2.assignment, updaters=updaters)

### Add constraints

In [14]:
popbound = within_percent_of_ideal_population(final_partition, 0.3)

### Setup Spectral Proposal

In [15]:
def spectral_cut(G):
    nlist = list(G.nodes())
    n = len(nlist)
    NLM = (nx.normalized_laplacian_matrix(G)).todense()
    # LM = (nx.laplacian_matrix(G)).todense()
    NLMva, NLMve = LA.eigh(NLM)
    NFv = NLMve[:, 1]
    xNFv = [NFv.item(x) for x in range(n)]
    node_color = [xNFv[x] > 0 for x in range(n)]

    clusters = {nlist[x]: node_color[x] for x in range(n)}

    return clusters


def propose_spectral_merge(partition):
    edge = random.choice(tuple(partition["cut_edges"]))
    # print(edge)
    et = [partition.assignment[edge[0]], partition.assignment[edge[1]]]
    # print(et)
    sgn = []
    for n in partition.graph.nodes():
        if partition.assignment[n] in et:
            sgn.append(n)

    # print(len(sgn))
    sgraph = nx.subgraph(partition.graph, sgn)

    edd = {0: et[0], 1: et[1]}

    # print(edd)

    clusters = spectral_cut(sgraph)
    # print(len(clusters))
    flips = {}
    for val in clusters.keys():
        flips[val] = edd[clusters[val]]

    # print(len(flips))
    # print(partition.assignment)
    # print(flips)
    return partition.flip(flips)

### Build and run final Markov chain

In [16]:
final_chain = MarkovChain(
    propose_spectral_merge,
    Validator([]),
    accept=always_accept,
    initial_state=final_partition,
    total_steps=25,
)


for part3 in final_chain:
    plt.figure()
    nx.draw(
        graph,
        pos,
        node_color=[part3.assignment[x] for x in graph.nodes()],
        node_size=ns,
        node_shape="s",
        cmap="tab20",
    )
    plt.savefig(f"./plots/medium/spectral_step{part3['step_num']:02d}.png")
    plt.close()

print("Finished Spectral")

Finished Spectral
